In [124]:
from db import *
import streamlit as st
import pandas as pd
import re
import random
import streamlit.components.v1 as components
from streamlit_player import st_player
from PIL import Image
import os
import flask
from flask import request, jsonify
from functrans import listToString,read_pdf,docx2txtrans,read_resumes,translator,skill_resumes,get_skjob_resum # skills_doc
#from préprocess import read_r,get_cleaned_words,remove_tags,html_parser,get_cleaned_wordsJ
import os
import io
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
import docx2txt
from docx import Document
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
import pandas as pd
import textwrap
from deep_translator import GoogleTranslator
import textwrap
from langdetect import detect
import warnings
import os
warnings.filterwarnings("ignore")
# Import libraries
#import sqlite3 as sql # sql connector
from datetime import datetime
import numpy as np             #for numerical computations like log,exp,sqrt etc
import pandas as pd            #for reading & storing data, pre-processing
import matplotlib.pylab as plt #for visualization
#import matplotlib.pyplot as plt
import re
from Cleaner import *
#from Cleaner import *
import tf_idf
exec(open("functrans.py").read())
exec(open("Cleaner.py").read())
from pyresparser import ResumeParser
import html.parser

import matplotlib.colors as mcolors
import gensim
import gensim.corpora as corpora
from gensim.summarization import keywords
from operator import index
from wordcloud import WordCloud
from pandas._config.config import options
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import Similar
from PIL import Image
import time
import pymysql
from flask import Flask, render_template, request
from flaskext.mysql import MySQL
from ast import literal_eval
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
import re
import regex

In [125]:
my_file = open("Foutfile.txt",encoding='utf-8')
content = my_file.read()
content_list = content.split("\n")

keywords1 = [re.sub(' +', ' ', e) for e in content_list]

keywords2 = [e.lower() for e in keywords1]
keywords2[:] = [item for item in keywords2 if item != '']
keywords2=list(map(lambda x: x.lower(), keywords2))

def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]


def skills_doc(dirty):
    clean = str(dirty).lower()
    splitclean=clean.split(' ')
    splitclean2=[x.strip("', /\n") for x in splitclean ]
    wordList = keywords2
    result = regex.findall(r"\L<words>", clean, words=keywords2)
    L1=f7([x for x in result if x in wordList]) #retenir les skills qui sont dans la liste de mot clé
    return f7([x for x in splitclean2 if x in L1])# retenir les skills qui sont dans le text par rapport à L1

#Néttoyage du document comportant les CV
#evité de stcache
def get_cleaned_words(document):
    for i in range(len(document)):
        raw = Cleaner(document[i][1])
        document[i].append(" ".join(raw[0]))
        document[i].append(" ".join(raw[1]))
        document[i].append(" ".join(raw[2]))
        try:
            sentence = tf_idf.do_tfidf(document[i][3].split(" "))
        except:
            sentence=Cleaner(document[i][3])
        document[i].append(sentence)
    return document

In [126]:
email="dahou.r@yahoo.com"

In [142]:
#Accédons au dossier contenant le CV des candidat qui ont postulé à l'offre
resume_dir = str(email+"cv")+"/"
resume_names = os.listdir(resume_dir)
#document = []
documentcv = read_resumes(resume_names, resume_dir)
# pour filtrer utilisateur dont le CV ne fera pas aumoins 36 mots
docselect=[]
for i in documentcv:
    S=i[1]
    K=S.split()
    if len(K)>=36:
        docselect.append(i)

Doccv = get_cleaned_words(docselect)

Resumes1 = pd.DataFrame(Doccv, columns=["Nom", "Context", "Cleaned", "Selective", "Selective_Reduced", "TF_Based"])
#recupération des skills
#dict_skill=[]
    


skdict=[]
for i in range(0,Resumes1.shape[0]):
    d=Resumes1['Cleaned'][i]
    dict_skill=skills_doc(d)
    skdict.append(dict_skill)



In [153]:
dataskill= Resumes1.copy()

def listToStringWithoutBrackets(list1):
    return str(list1).replace('[','').replace(']','').replace("'",'')

In [157]:
dataskill= Resumes1.copy()
dataskill['Skills']=pd.DataFrame({'Skills': skdict})
dataskill['Skills']=dataskill['Skills'].apply(lambda row: listToStringWithoutBrackets(row))
dataskill

,Nom,Context,Cleaned,Selective,Selective_Reduced,TF_Based,Skills
0,ABHAY RAJ DWIVEDI.docx,ABHAY RAJ DWIVEDIFeatured Resume Software Deve...,ABHAY RAJ dwivedifeature Resume Software Devel...,Integrations development api get day SQL HTML ...,development api day SQL HTML Dec assist subsid...,aapnetadonetcsshtmljavascriptdevexpresssql abh...,"resume, sql, programming, science, email, educ..."
1,Abhishek Sharma.docx,Abhishek SharmaFeatured Resume Developer seeki...,Abhishek SharmaFeatured Resume Developer seeki...,get day computer airflow SQL utility Dec Profi...,day computer airflow SQL utility Dec Profile l...,abhishek active airflow area article automate ...,"resume, email, education, linux, hadoop, sql, ..."
2,ADRYJA GHOSH.docx,ADRYJA GHOSHFeatured Resume English Teacher se...,ADRYJA GHOSHFeatured Resume English Teacher se...,interactive Open Other physical provide teachi...,interactive provide teaching atmosphere Summar...,academic academy accommodate active administra...,"resume, english, teaching, education, email, c..."
3,AJAY CHINNI.docx,AJAY CHINNIFeatured Resume Seeking roles in Ma...,AJAY chinnifeature Resume seek role Machine Le...,Image api patient computer prototype Processin...,Image api computer prototype Fullstack datum S...,863c41556ec744d7b178c3c114e76e35last 9sep20 ac...,"resume, engineering, programming, email, educa..."
4,ALANKRIT NIRJHAR.docx,ALANKRIT NIRJHARPremium CV Featured Resume Sen...,ALANKRIT NIRJHARPremium CV Featured Resume Sen...,Image acquire flatten day computer clean Other...,Image acquire flatten day computer SQL realtim...,accuracy achieve acquire active actively alank...,"cv, resume, mathematics, analytics, research, ..."
...,...,...,...,...,...,...,...
57,Suraj Chauhan.docx,Suraj Chauhan Senior Web Developer seeking rol...,Suraj Chauhan Senior web Developer seek role W...,api Proficient Dec party Senior pref HTML AWS ...,Proficient Dec party pref AWS Summary Jul Link...,21c5d05765c7180c58last 45xx active ad ajax als...,"programming, email, education, management, api..."
58,Swati Mehta.docx,Swati MehtaFeatured Resume Sales & Marketing H...,Swati MehtaFeatured Resume Sales Marketing Hea...,development improvement Head Wonder call Other...,development improvement Head Wonder salary Dec...,accord account achieve action active activity ...,"resume, sales, marketing, management, economic..."
59,Vikram Balasubramanian.docx,Vikram BalasubramanianFeatured Resume National...,Vikram BalasubramanianFeatured Resume National...,store Bandra sector presence Known promotion g...,store Bandra sector presence promotion territo...,48078373777b2b7f21last abrl account accounts a...,"resume, sales, email, education, business, adm..."
60,Vipin Kumar.docx,Vipin KumarFeatured Resume HR Manager seeking ...,Vipin KumarFeatured Resume HR Manager seek ass...,offer day ManagementAdministrationRecruitmentE...,day ManagementAdministrationRecruitmentEmploye...,59d67f6e197785c024last account acquisitiontrai...,"resume, management, administration, email, edu..."


In [158]:

Resumes1.reset_index(drop=True, inplace=True)
dataskill.reset_index(drop=True, inplace=True)

Resumes=pd.concat( [Resumes1, dataskill[['Skills']]], axis=1) 
#Database.to_csv("Resume_Data.csv", index=False)
#os.system("rm -rf"+" "+email+"cv")

#Accédons au dossier contenant du Job
jobs_dir = str(email+"jb")+"/"
jobs_names = os.listdir(jobs_dir)
#traduction et ajout dans un doc
#document = []
documentjb = read_resumes(jobs_names, jobs_dir)
Docjb= get_cleaned_words(documentjb)

Jobs = pd.DataFrame(Docjb, columns=["Nom", "Context", "Cleaned", "Selective", "Selective_Reduced", "TF_Based"])

In [159]:
Jobs

,Nom,Context,Cleaned,Selective,Selective_Reduced,TF_Based
0,Backend Developer.docx,Backend Developer - Python/Django We are looki...,Backend Developer PythonDjango look PythonDjan...,development api control HTML party sense Front...,development api control HTML party sense Front...,ability able additional aesthetic analytic ana...
1,Web Developer.docx,Web Developer We are looking for an outstandin...,web Developer look outstanding web Developer r...,development computer HTML client diverse datum...,development computer HTML client datum session...,ability activity adequate application apply ba...


In [160]:
def calculate_scores(resumes, job_description):
    scores = []
    for x in range(resumes.shape[0]):
        score = Similar.match(
            resumes['TF_Based'][x], job_description['TF_Based'][index])
        scores.append(score)
    return scores

index=0
Resumes['Scores'] = calculate_scores(Resumes, Jobs)
Ranked_resumes=Resumes

Ranked_resumes = Resumes.sort_values(
    by=['Scores'], ascending=False).reset_index(drop=True)

Ranked_resumes['Rang'] = pd.DataFrame(
    [i for i in range(1, len(Ranked_resumes['Scores'])+1)])

In [164]:
job_skill=skills_doc(Jobs['Cleaned'][index])

In [165]:
job_skill

['python',
 'django',
 'api',
 'numpy',
 'storage',
 'architecture',
 'design',
 'postgresql',
 'environment',
 'writing',
 'communication',
 'integration',
 'javascript',
 'jquery',
 'salary',
 'education']

In [166]:
 ##Comparaisons Skill Job VS Skill candidat
words = job_skill
stopwords =["Email,","Email","email,","email","Gmail,","Gmail","Yahoo,","Yahoo","gmail,","gmail","yahoo,","yahoo","Outlook,","Outlook","outlook,","outlook","Telephone,","Telephone","telephone,","telephone"]
wordsf=[]
for x in words:
    if x not in stopwords:
        wordsf.append(x)

countlstJ=[]
interlstJ=[]

globalskills=wordsf #skills job
globalskills=[x.strip("', /\n") for x in globalskills ]
for i in range(0,dataskill.shape[0]):
    listskills=dataskill['Skills'][i].split()
    listskills=[x for x in listskills if str(x) != 'nan'] # pour supprimer les valeur nuls
    listskills=[x.split(',') for x in listskills]
    listskills = list({x.strip("', /\n") for l in listskills for x in l})
    loacalskills = [x.replace('\\xa0', '').replace(':', '').replace('"', '') for x in listskills]
    #inter=set(globalskills).intersection(loacalskills)
    inter=list(set([x.lower() for x in globalskills]) & set([x.lower() for x in loacalskills]))
    interlstJ.append(inter)
    count = len(inter)
    countlstJ.append(count)

#Creation de la dataframe Nom et skill

comptsklJ = pd.DataFrame({'skills_e': interlstJ,'count': countlstJ})
comptsklJ['Noms']=Resumes['Nom']
comptsklJ.head()



stdictJ=[]
# convert to string
for i in range(0,comptsklJ.shape[0]):
    input= str(comptsklJ['skills_e'][i]).replace('{','').replace('}','').replace(",'","'")
    stdictJ.append(input)

comptsklJ['skills_e'] = pd.DataFrame({'kills_e':stdictJ})
comptsklJ['score']=comptsklJ['count'].astype('int')/len(globalskills)*100
#comptsklJ.sort_values(by=['score'], ascending=False,inplace=True)

In [174]:
Ranked_resumes2=pd.merge(sj_sc.rename(columns = {'Noms': 'Nom'}, inplace = False), Ranked_resumes[['Nom','Cleaned','Skills']] , how='left',on="Nom")

In [176]:
comptsklJ

,skills_e,count,Noms,score
0,"['api', 'design', 'education', 'javascript']",4,ABHAY RAJ DWIVEDI.docx,25.00
1,"['python', 'education']",2,Abhishek Sharma.docx,12.50
2,['education'],1,ADRYJA GHOSH.docx,6.25
3,"['django', 'api', 'education', 'python', 'desi...",5,AJAY CHINNI.docx,31.25
4,"['python', 'education']",2,ALANKRIT NIRJHAR.docx,12.50
...,...,...,...,...
57,"['api', 'jquery', 'education', 'integration', ...",5,Suraj Chauhan.docx,31.25
58,"['salary', 'education']",2,Swati Mehta.docx,12.50
59,"['education', 'design']",2,Vikram Balasubramanian.docx,12.50
60,"['salary', 'education']",2,Vipin Kumar.docx,12.50


In [136]:
listskills

['None']